# Сборный проект-2

Вы работаете в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи вашего мобильного приложения.

Изучите воронку продаж. Узнайте, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?

После этого исследуйте результаты A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/B-теста. Для него пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясните, какой шрифт лучше.

В случае общей аналитики и A/B-эксперимента работайте с одними и теми же данными. В реальных проектах всегда идут эксперименты. Аналитики исследуют качество работы приложения по общим данным, не учитывая принадлежность пользователей к экспериментам.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from scipy import stats as st
import numpy as np
import statsmodels.stats.api as sms
import math as mth

In [2]:
df = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
df

FileNotFoundError: [Errno 2] File /datasets/logs_exp.csv does not exist: '/datasets/logs_exp.csv'

В первоначальном виде данные были склеяны.Был добавлен разделитель "\t". Теперь подготовим данные к анализу, займемся их предобработкой.

Таблица df. Значения наименования столбцов:

    1) EventName - название события
    2) DeviceIDHash - уникальный идентификатор пользователя
    3) EventTimestamp - время события
    4) ExpId - номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная

## Шаг 2. Подготовьте данные

Заменим названия столбцов.

In [ ]:
df.columns = ['event', 'user_id', 'event_time', 'exp_num']

Теперь проверим пропуски и типы данных.

In [ ]:
df.info()

Теперь проверим пропуски, дубликаты и типы данных.

In [ ]:
df.isnull().sum()

Пропусков не обнаружено

In [ ]:
df.duplicated().sum()

Посмотрим на дубликаты

In [ ]:
df.loc[df.duplicated()]

Как видим, строчки не полностью совпадают, поэтому удалять их не нужно.

Преобразуем время в формат datetime64[ns]

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'], unit='s')
df

In [ ]:
df.info()

Добавим столбец с датами

In [ ]:
df['event_date'] = df['event_time'].dt.strftime('%Y-%m-%d')
df

И преобразуем в формат datetime64[ns]

In [ ]:
df['event_date'] = df['event_date'].astype('datetime64[ns]')
df.info()

***Итак, данные готовы для дальнейшего анализа. Была проведена их полная предобработка. Были заменены форматы на релевантные, добавлены столбцы с датой и временем, проверены дубликаты и наличие нулей в данных. Можно проверять данные дальше.***

## Шаг 3. Изучите и проверьте данные

- Сколько событий в логе?

In [ ]:
df['event'].count()

Количество уникальных событий

In [ ]:
df['event'].unique()

- Сколько пользователей в логе?

In [ ]:
df['user_id'].nunique()

- Сколько в среднем событий приходится на пользователя?

In [ ]:
df['event'].count() / df['user_id'].nunique()

- Данными за какой период вы располагаете? Найдите максимальную и минимальную дату. Постройте гистограмму по дате и времени. Можно ли быть уверенным, что у вас одинаково полные данные за весь период? Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Определите, с какого момента данные полные и отбросьте более старые. Данными за какой период времени вы располагаете на самом деле?

In [ ]:
df['event_time'].describe()

Располагаем данными с 2019-07-25 по 2019-08-07

In [ ]:
df.groupby([df['event_date']]).count().plot(kind="bar", y = 'event')

Судя по распределению частот, до 1 августа 2019 года данные были неполные. Целесообразно для проведения дальнейшего анализа сделать срез по этой дате, поскольку фактически мы обладаем данными с этого момента.

In [ ]:
new_df = df.query('event_date >= "2019-08-01"').reset_index(drop=True)
new_df

- Много ли событий и пользователей вы потеряли, отбросив старые данные?

In [ ]:
deleted_df = 1 - len(new_df) / len(df)

In [ ]:
print ('Удалили {:.1%} данных'.format(deleted_df))

In [ ]:
new_df['event'].count()

In [ ]:
new_df['user_id'].nunique()

In [ ]:
events = ((df['event'].count() - new_df['event'].count())/df['event'].count())
print('Событий потеряли:','{:.2%}'.format(events))

In [ ]:
users = ((df['user_id'].nunique() - new_df['user_id'].nunique())/df['user_id'].nunique())
print('Пользователей потеряли:','{:.2%}'.format(users))

- Проверьте, что у вас есть пользователи из всех трёх экспериментальных групп.

In [ ]:
new_df['exp_num'].nunique()

***Итак, 1.16% событий и 0.23% пользователей были потеряны в результате среза. Потери незначительны.***

In [ ]:
# Выделим данные в отдельные датафреймы разбитые по событиям:
tut_df = new_df[new_df['event']=='Tutorial']
main_df = new_df[new_df['event']=='MainScreenAppear']
offer_df = new_df[new_df['event']=='OffersScreenAppear']
cart_df = new_df[new_df['event']=='CartScreenAppear']
payment_df = new_df[new_df['event']=='PaymentScreenSuccessful']
offer_df

In [ ]:
# Найдём тех пользователей, чьи id присутствуют в предыдущих этапах воронки:
step1 = main_df[~main_df['user_id'].isin(tut_df)]
step2 = offer_df[~(offer_df['user_id'].isin(main_df) & offer_df['user_id'].isin(tut_df))]
step3 = cart_df[~(cart_df['user_id'].isin(main_df) & cart_df['user_id'].isin(offer_df) & cart_df['user_id'].isin(tut_df))]
step4 = payment_df[~(
    payment_df['user_id'].isin(tut_df) &
    payment_df['user_id'].isin(main_df) & 
    payment_df['user_id'].isin(offer_df) & 
    payment_df['user_id'].isin(cart_df))]
step2

In [ ]:
# Из получившихся датафреймов соберём "настоящую" воронку:
data_clean_funnel = pd.concat([tut_df, step1, step2, step3, step4])
data_clean_funnel

In [ ]:
# Сгруппируем чтобы посмотреть сколько пользователей прошло совершили все шаги:
data_clean_funnel_real = data_clean_funnel.groupby(['event','exp_num']).agg({'user_id':'nunique'}).reset_index().sort_values(['exp_num', 'user_id'], ascending=False)
data_clean_funnel_real


In [ ]:
data_clean_funnel_real = data_clean_funnel_real.query('event != "Tutorial"')

In [ ]:
# Соберём data_clean_funnel_real в табличный вид:
data_clean_funnel_real_pivot = pd.pivot_table(data_clean_funnel_real, 
               index='event',
               columns='exp_num', 
               values='user_id').sort_values(246,ascending=False)
data_clean_funnel_real_pivot

Видим что в фрейме имеются пользователи из всех трех групп.

## Шаг 4. Изучите воронку событий

- Посмотрите, какие события есть в логах, как часто они встречаются. Отсортируйте события по частоте.

In [ ]:
new_df.groupby('event')['event'].count().sort_values(ascending=False)

В логах имеются следующие события: "Главный экран","Предложения", "Положили в корзину", "Оплатили", "Прошли обучение"

- Посчитайте, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитайте долю пользователей, которые хоть раз совершали событие.

In [ ]:
pivot_df = new_df.pivot_table(index='event', values='user_id', aggfunc='nunique').sort_values('user_id', ascending=False)
pivot_df

In [ ]:
pivot_df['count_id'] = new_df.groupby('event')['event'].count().sort_values(ascending=False)
pivot_df

In [ ]:
pivot_df['part']=pivot_df['user_id']/new_df['event'].count()*100
pivot_df

- Предположите, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки.

Предполагаю, что события идут в следующем порядке: Tutorial → MainScreenAppear → OffersScreenAppear → CartScreenAppear → PaymentScreenSuccessful.
Логично предположить, что Tutorial происходит в начале, но не все предпочитают его проходить, поэтому его исключим из воронки.

In [ ]:
pivot_df = pivot_df.query('user_id != 840')
pivot_df

- По воронке событий посчитайте, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C, посчитайте отношение числа пользователей с событием B к количеству пользователей с событием A.

In [ ]:
first_step = pivot_df['user_id'][1] / pivot_df['user_id'][0]
second_step = pivot_df['user_id'][2] / pivot_df['user_id'][1]
third_step = pivot_df['user_id'][3] / pivot_df['user_id'][2]

In [ ]:
print ('Конверсия из MainScreenAppear в OffersScreenAppear:', '{:.2%}'.format(first_step))
print ('Конверсия из OffersScreenAppear в CartScreenAppear:', '{:.2%}'.format(second_step))
print ('Конверсия из CartScreenAppear в PaymentScreenSuccessful:', '{:.2%}'.format(third_step))

Всю ту же информацию можно поглядеть на визуализации воронки, которую я расположу чуть ниже

Переименуем колонки и добавим колонку с суммарным количеством пользователей контрольных групп

In [ ]:
data_clean_funnel_real_pivot.columns = ['A1', 'A2', 'B']
data_clean_funnel_real_pivot['mixed_AA'] = data_clean_funnel_real_pivot['A1'] + data_clean_funnel_real_pivot['A2']
final_df = data_clean_funnel_real_pivot[['A1', 'A2', 'mixed_AA', 'B']]
final_df

In [ ]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Воронка группы А1',
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['A1'],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Воронка группы А2',
    orientation = "h",
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['A2'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Воронка группы B',
    orientation = "h",
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['B'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.show()

мы получили воронку событий в разрезе трех групп: А1, А2, и B

- На каком шаге теряете больше всего пользователей?

Невооруженным глазом видно что большая часть пользователей теряется на самом первом шаге, при переходе с главного экрана к экрану с товарами. На этом шаге мы терям 40% пользователей. Около 20% пользователей уходят на втором этапе - этапе перехода от товаров к корзине с покупками. Ну и самые незначительные потери мы видим на этапе перехода от корзины к оплате, тут мы теряем 5% пользователей.

- Какая доля пользователей доходит от первого события до оплаты?

In [ ]:
part_of_total = pivot_df['user_id'][3] / pivot_df['user_id'][0]
print ('Итоговая конверсия из посетителя в покупателя:', '{:.2%}'.format(part_of_total))

***Во всем датафрейме чуть меньше половины доходит до оплаты, а теряется больше всего пользователей после первого шага. Возможно это связано с тем, что пользователи случайно оказались на главном экране.***

## Шаг 5. Изучите результаты эксперимента

- Сколько пользователей в каждой экспериментальной группе?

In [ ]:
groups = new_df.pivot_table(index='exp_num', values='user_id', aggfunc='nunique')
groups

Чисто визуально количество униклаьных пользователей в группе практически не отличается. Нужно будет знать, статистически значимы ли отличия между этими группами.

- Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.

Выведем снова конечную воронку, чтобы для расчета Z статистики брать оттуда значения.


In [ ]:
final_df

Подставим теперь значения для групп А1 и А2 для проверки равенства долей. Предположим, что доли пользователей, которые совершают события у групп А1 и А2 равны. Альтернативная гипотеза будет утверждать, что разница между долями значима.

In [ ]:
alpha = 0.05

purchases = np.array([1200, 1158])
leads = np.array([2484, 2513])

# пропорция успехов в первой группе:
p1 = purchases[0] / leads[0]

# пропорция успехов во второй группе:
p2 = purchases[1] / leads[1]

# пропорция успехов в комбинированном датасете:
p_combined = (purchases[0] + purchases[1]) / (leads[0] + leads[1])

# разница пропорций в датасетах
difference = p1 - p2

# считаем статистику в ст.отклонениях стандартного нормального распределения
z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / leads[0] + 1 / leads[1]))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1)

p_value = (1 - distr.cdf(abs(z_value))) * 2

print('p-значение: ', p_value)

if (p_value < alpha):
    print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
else:
    print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

Итак, статистика показала, что доли групп А1 и А2 нельзя считать разными.

- Выберите самое популярное событие. Посчитайте число пользователей, совершивших это событие в каждой из контрольных групп. Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?

Самым популярным событием у пользователей является просмотр главного экрана.

In [ ]:
def compare(data):
    for k in range(len(data)-3):
        print('')
        print('Для', k+1,'контрольной группы и контрольной группы 2')
        print('--------------------------------------')
        for i in range(len(data)-1): 
            
            alpha = .05 
            
            successes = np.array(list(data.iloc[i+1,[k,1]]))
            trials = np.array(list(data.iloc[i,[k,1]]))
            
            p1 = successes[0]/trials[0]
            p2 = successes[1]/trials[1]
            
            p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
            
            difference = p1 - p2
            
            z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
            distr = st.norm(0, 1) 
            
            p_value = (1 - distr.cdf(abs(z_value))) * 2
            
            print('p-значение: ', p_value)
            if (p_value < alpha):
                print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
            else:
                print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")
    return

In [ ]:
compare(final_df)

По полученным данным видно, что для контрольных групп А1 и А2 нет статистически значимых различий между долями, значит можно утверждать, что разбиение на контрольные группы работает корректно

Рассчитаем число пользователей, совершивших это событие в каждой из контрольных групп.

In [ ]:
print('Число пользователей, совершивших событие в группе А1:',final_df.iloc[0][0])

In [ ]:
print('Число пользователей, совершивших событие в группе А2:',final_df.iloc[0][1])

Рассчитаем долю пользователей, совершивших это событие в группах А1 и А2.

In [ ]:
part_a1 = final_df.iloc[0][0]/pivot_df['user_id'][0]
print('{:.2%}'.format(part_a1)) 

In [ ]:
part_a2 = final_df.iloc[0][1]/pivot_df['user_id'][0]
print('{:.2%}'.format(part_a2)) 

Доли пользователей, совершивших самое популярное событие, в контрольных группах А1 и А2 не имеют статистически значимых различий.

- Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?

Для этих целей используем функцию, которой нужно передать конечный датафрейм с воронкой. В теле функции зададим два цикла, которые построчно бы сравнивали поочередно колонку группы А1 с колонкой группы B, затем колонку группы А2 с колонкой группы B и, наконец, колонку сборной группы А1 и А2 с колонкой группы B. Аналогично, в качестве нулевой гипотезы зададим равенство долей пользователей в двух сравниваемых группах, а качестве альтернативной гипотезы – разница между долями пользователей, совершающих события, в двух разных группах значимо различаются.

In [ ]:
def func_1(data):
    for k in range(len(data)-1):
        print('')
        print('Для', k+1,'контрольной группы и тестовой')
        print('--------------------------------------')
        for i in range(len(data)-1):
            alpha = .05 
            successes = np.array(list(data.iloc[i+1,[k,3]]))
            trials = np.array(list(data.iloc[i,[k,3]]))
            p1 = successes[0]/trials[0]
            p2 = successes[1]/trials[1]
            p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
            difference = p1 - p2
            z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
            distr = st.norm(0, 1) 
            p_value = (1 - distr.cdf(abs(z_value))) * 2
            print('p-значение: ', p_value)
            if (p_value < alpha):
                print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
            else:
                print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")
    return

In [ ]:
func_1(final_df)

***Таким образом, между контрольной группой А2 и экспериментальной группой B на 3 шаге воронки (при переходе от корзины к оплате) есть небольшая статистическая разница в результате чего отвергается нулевая гипотеза. При этом, на всех остальных шагах нет статистически значимых событий между шагами. Ввиду этого, следует сделать вывод о том, что такими небольшими статистически значимыми различиями в тесте можно пренебречь. Таким образом, нет необходимости в введении нового шрифта для пользователей.***

- Какой уровень значимости вы выбрали при проверке статистических гипотез выше? Посчитайте, сколько проверок статистических гипотез вы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если вы хотите изменить его, проделайте предыдущие пункты и проверьте свои выводы.

Так как у нас происходило 3 сравнения, то критический уровень статистической значимости 0,05 был скорректирован с помощью порпавки Бонферрони:

In [ ]:
def func_2(data):
    for k in range(len(data)-1):
        print('')
        print('Для', k+1,'контрольной группы и тестовой')
        print('--------------------------------------')
        for i in range(len(data)-1):
            alpha = .05
            bonferroni_alpha = alpha / 3
            successes = np.array(list(data.iloc[i+1,[k,3]]))
            trials = np.array(list(data.iloc[i,[k,3]]))
            p1 = successes[0]/trials[0]
            p2 = successes[1]/trials[1]
            p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
            difference = p1 - p2
            z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
            distr = st.norm(0, 1) 
            p_value = (1 - distr.cdf(abs(z_value))) * 2
            print('p-значение: ', p_value)
            if (p_value < bonferroni_alpha):
                print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
            else:
                print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")
    return

In [ ]:
func_2(final_df)

***Таким образом, даже при смене статистически значимого критерия альфа, на каждом шагу для каждой группы мы получили почти такой же результат по принятию или отвержению нулевой гипотезы. Это означает, что наши данные верные и судя по A/B тесту нет необходимости во введении нового шрифта в приложении.***

## Общие выводы

После произведеноого анализа и проведения экспериментов можно сделать следующие выводы:
 - В среднем на одного пользователя приходится 32,3 события. 
 - Видно что большая часть пользователей теряется на самом первом шаге, при переходе с главного экрана к экрану с товарами. На этом шаге мы терям 40% пользователей. Около 20% пользователей уходят на втором этапе - этапе перехода от товаров к корзине с покупками. Ну и самые незначительные потери мы видим на этапе перехода от корзины к оплате, тут мы теряем 5% пользователей.
 - Итоговая конверсия из посетителя в покупателя: 47.70%
 - Обучающим разделом (tutorial), который должен по логике находиться на самом верху воронки, воспользовалось меньше всего пользователей. Скорее всего большинство его просто пропускало.
 - Нет основание считать, что показатели экспериментальной группы B с изменённым шрифтом отличаются от групп с первоначальным вариантом шрифта по всем событиям. Из этого следует, что внедрение нового типа шрифта протестированного на группе B не приведет к увеличению показателей.

In [ ]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Воронка группы А1',
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['A1'],
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Воронка группы А2',
    orientation = "h",
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['A2'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'Воронка группы B',
    orientation = "h",
    y = ["Главный экран","Предложения", "Положили в корзину", "Оплатили"],
    x = final_df['B'],
    textposition = "inside",
    textinfo = "value+percent initial"))

fig.show()